d-sandbox

<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 600px; height: 163px">
</div>

# Lab: Packaging a Project

## ![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) In this lab you:<br>
- Define an MLProject file
- Define a Conda environment
- Define your machine learning script
- Execute your solution as a run

In [3]:
%run "./../Includes/Classroom-Setup"

## Defining the MLproject file

Write an MLproject file called `MLproject` to the path defined for you below.

In [5]:
path = userhome + "/ml-production/03-lab/"

dbutils.fs.rm(path, True) # Clears the directory if it already exists
dbutils.fs.mkdirs(path)

print("Created directory `{}` to house the project files.".format(path))

Created directory `dbfs:/user/shashank.rao@rhsmith.umd.edu/ml-production/03-lab/` to house the project files.

The file should consist of the following aspects:<br><br>

0. The name should be `Lab-03`
0. It should use the environment `conda.yaml`
0. It should take the following parameters:
   - `data_path`: a string with a default of `/dbfs/mnt/training/airbnb/sf-listings/airbnb-cleaned-mlflow.csv`
   - `bootstrap`: a boolean with a default of `True`
   - `min_impurity_decrease`: a float with a default of `0.`
0. The command that uses the parameters listed above

In [7]:
#  TODO
dbutils.fs.put(path + "MLproject", 
'''
name: Lesson-3-Lab-Model-Training

conda_env: conda.yaml

entry_points:
  main:
    parameters:
      data_path: {type: str, default: "/dbfs/mnt/training/airbnb/sf-listings/airbnb-cleaned-mlflow.csv"}
      bootstrap: {type: bool, default: True}
      min_impurity_decrease: {type: float, default: 0.}
    command: "python train.py --data_path {data_path} --bootstrap {bootstrap} --min_impurity_decrease {min_impurity_decrease}"
'''.strip())

Wrote 428 bytes.
Out[10]: True

## Defining the Environment

Define the conda environment.  It should include the following libraries:<br><br>

  - `cloudpickle=0.5.3`
  - `numpy=1.14.3`
  - `pandas=0.23.0`
  - `scikit-learn=0.19.1`
  - `pip:`
    - `mlflow==1.0.0`

In [9]:
#  TODO
dbutils.fs.put(path + "conda.yaml", 
'''
name: Lesson-03-Lab
channels:
  - defaults
dependencies:
  - cloudpickle=0.5.3
  - numpy=1.14.3
  - pandas=0.23.0
  - scikit-learn=0.19.1
  - pip:
    - mlflow==1.0.0
'''.strip())

Wrote 166 bytes.
Out[11]: True

## Defining the Machine Learning Script

Based on the script from Lesson 3, create a Random Forest model that uses the parameters `data_path`, `bootstrap`, and `min_impurity_decrease`.

In [11]:
#  TODO
dbutils.fs.put(path + "train.py", 
'''
import click
import mlflow.sklearn
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split

@click.command()
@click.option("--data_path", default="/dbfs/mnt/training/airbnb/sf-listings/airbnb-cleaned-mlflow.csv", type=str)
@click.option("--bootstrap", default=True, type=bool)
@click.option("--min_impurity_decrease", default=0., type=float)

def mlflow_rf(data_path, bootstrap, min_impurity_decrease):

  with mlflow.start_run() as run:
    # Import the data
    df = pd.read_csv(data_path)
    X_train, X_test, y_train, y_test = train_test_split(df.drop(["price"], axis=1), df[["price"]].values.ravel(), random_state=42)
    
    # Create model, train it, and create predictions
    rf = RandomForestRegressor(bootstrap=bootstrap, min_impurity_decrease=min_impurity_decrease)
    rf.fit(X_train, y_train)
    predictions = rf.predict(X_test)

    # Log model
    mlflow.sklearn.log_model(rf, "random-forest-model")
    
    # Log params
    mlflow.log_param("bootstrap", bootstrap)
    mlflow.log_param("min_impurity_decrease", min_impurity_decrease)

    # Log metrics
    mlflow.log_metric("mse", mean_squared_error(y_test, predictions))
    mlflow.log_metric("mae", mean_absolute_error(y_test, predictions))  
    mlflow.log_metric("r2", r2_score(y_test, predictions))  

if __name__ == "__main__":
    mlflow_rf() # Note that this does not need arguments thanks to click 
'''.strip())

Wrote 1522 bytes.
Out[12]: True

## Executing your Solution

First make sure that the three necessary files are where they need to be.

In [13]:
dbutils.fs.ls(path)

Out[13]: [FileInfo(path='dbfs:/user/shashank.rao@rhsmith.umd.edu/ml-production/03-lab/MLproject', name='MLproject', size=428),
 FileInfo(path='dbfs:/user/shashank.rao@rhsmith.umd.edu/ml-production/03-lab/conda.yaml', name='conda.yaml', size=166),
 FileInfo(path='dbfs:/user/shashank.rao@rhsmith.umd.edu/ml-production/03-lab/train.py', name='train.py', size=1522)]

Execute your solution with the following code.

In [15]:
import mlflow

mlflow.projects.run(uri=path.replace("dbfs:","/dbfs"),
  parameters={
    "data_path": "/dbfs/mnt/training/airbnb/sf-listings/airbnb-cleaned-mlflow.csv",
    "bootstrap": False,
    "min_impurity_decrease": .1
})

Out[14]: <mlflow.projects.submitted_run.LocalSubmittedRun at 0x7f4d95fe4c50>

-sandbox
## ![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Next Lesson<br>

<img alt="Side Note" title="Side Note" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.05em; transform:rotate(15deg)" src="https://files.training.databricks.com/static/images/icon-note.webp"/> See the solutions folder for an example solution to this lab.

### [Start the next lesson, Model Management.]($../04-Model-Management )

-sandbox
&copy; 2019 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="http://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="http://help.databricks.com/">Support</a>